In [35]:
import pandas as pd
import os
import torch
import json
import numpy as np

In [36]:
import matplotlib.pyplot as plt

In [37]:
%run ../Config.ipynb

In [38]:
config = Config()

#### Set sent to be docked 

In [55]:
refined_set_reduced = pd.read_csv('../Datahub/pdbs_refined_set.csv')

In [40]:
len(refined_set_reduced)

4790

In [41]:
"4eor" in list(refined_set['PDB code'])

False

#### pdbs that were meant to be docked 

#### PDbs docked 

In [42]:
import os

def find_pdb_with_results(path):
    # Get the list of pdb folders in the specified path
    pdb_folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]

    # Initialize list to store pdb folders with 'results'
    pdb_with_results = []

    for pdb_folder in pdb_folders:
        pdb_path = os.path.join(path, pdb_folder)
        results_path = os.path.join(pdb_path, 'results')

        # Check if the 'results' folder exists in the pdb folder
        if os.path.exists(results_path) and os.path.isdir(results_path):
            pdb_with_results.append(pdb_folder)

    return pdb_with_results

# Specify the path to the 'set' folder
set_folder_path = '../set'

# Call the function to find pdb folders with 'results'
result_pdbs = find_pdb_with_results(set_folder_path)

#### QTD pdbs docked 

In [43]:
len(result_pdbs)

3998

In [44]:
"5qa8" in result_pdbs

False

#### QTD Pdbs com rmsd calculdados com sucesso 

In [45]:
rmsd = pd.read_csv(f"{config.data}/rmsd_refined_set.csv") # rmsd

In [46]:
rmsd['pdb'].nunique()

3940

#### QTd pdbs com menos de 10 poses 

In [47]:
import seaborn as sns 

In [48]:
rmsd_pose_count = rmsd.groupby('pdb')['poserank'].nunique().sort_values().reset_index()

In [49]:
rmsd_pose_count[rmsd_pose_count['poserank'] !=10 ]['pdb'].nunique()

952

### Binana 

#### QTD PDbs que o binana gerou resultados 

#### Tempo para rodar por quantidade de pdbs 

284 - PDBs - 29 min!

In [59]:
pdb = "5qa8"

In [60]:
pdb in list(refined_set_reduced['PDB code'])

True

In [61]:
pdb in result_pdbs

False